In [108]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import os
import glob

In [2]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('lab5') \
                    .getOrCreate()

In [70]:
df = spark.read.csv('Dataset/airports.csv', inferSchema=True, header=True)


In [71]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- LATITUDE: integer (nullable = true)
 |-- LONGITUDE: integer (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- REGION: string (nullable = true)



In [82]:
cpu=2


In [73]:
df_new = df.repartition(cpu)

In [113]:
df_new.groupBy("Country").count().selectExpr("Country","count as Airports").sort(col("count").desc()).limit(1).toPandas().to_csv("p2.csv")

In [114]:
df_new.groupby("Country").count().withColumnRenamed("count","Airports").coalesce(1).write.format("com.databricks.spark.csv").option("header", True).save("mydata1")
for file in glob.glob("mydata1/*.csv"):
    os.rename(file,"p1.csv")
import shutil
shutil.rmtree("mydata1", ignore_errors=True)

In [135]:
df_new.select("Name","Latitude","Longitude").filter("(Latitude between 10 and 90) and (Longitude between -90 and -10)").coalesce(1).write.format("com.databricks.spark.csv").option("header", True).save("mydata3")
for file in glob.glob("mydata3/*.csv"):
    os.rename(file,"p3.csv")
import shutil
shutil.rmtree("mydata3", ignore_errors=True)

In [136]:
data = pd.read_csv("p3.csv")

count    12589.000000
mean       -79.527524
std         10.300921
min        -90.000000
25%        -86.000000
50%        -81.000000
75%        -76.000000
max        -10.000000
Name: Longitude, dtype: float64